In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Reference: Bert Tokenizer + Fine-Tuning Model
https://www.kaggle.com/code/theodorospsarras/bert-tokenizer-fine-tuning-model/input

In [ ]:
#!pip install torch

In [1]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel,AutoModelForSequenceClassification
import pandas as pd
import torch.nn.functional as F
import numpy as np

In [ ]:
#import os

#os.environ['CUDA_VISIBLE_DEVICES']  = "0"

#torch.cuda.device_count()

In [2]:
prompts_train = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv")
prompts_test = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv")

summaries_train = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv")
summaries_test = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv")

sample_submission = pd.read_csv("/kaggle/input/commonlit-evaluate-student-summaries/sample_submission.csv")

prompts_train

,prompt_id,prompt_question,prompt_title,prompt_text
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...
1,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
2,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
3,ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."


1. prompt 파일과 train/test 파일 merge 하기

In [3]:
# dataframe 병합, prompt id 기준으로 각 아이디에 맞는 prompts_train, test 내용을 왼쪽에 붙임
train = summaries_train.merge(prompts_train, how="left", on="prompt_id")
test = summaries_test.merge(prompts_test, how="left", on="prompt_id")

train

,student_id,prompt_id,text,content,wording,prompt_question,prompt_title,prompt_text
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
...,...,...,...,...,...,...,...,...
7160,ff7c7e70df07,ebad26,They used all sorts of chemical concoctions to...,0.205683,0.380538,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."
7161,ffc34d056498,3b9047,The lowest classes are slaves and farmers slav...,-0.308448,0.048171,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
7162,ffd1576d2e1b,3b9047,they sorta made people start workin...,-1.408180,-0.493603,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
7163,ffe4a98093b2,39c16e,An ideal tragety has three elements that make ...,-0.393310,0.627128,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...


In [4]:
test

,student_id,prompt_id,text,prompt_question,prompt_title,prompt_text
0,000000ffffff,abc123,Example text 1,Summarize...,Example Title 1,Heading\nText...
1,111111eeeeee,def789,Example text 2,Summarize...,Example Title 2,Heading\nText...
2,222222cccccc,abc123,Example text 3,Summarize...,Example Title 1,Heading\nText...
3,333333dddddd,def789,Example text 4,Summarize...,Example Title 2,Heading\nText...


In [6]:
train['merged_text'] = 'text '+train['text'] + ' prompt_question ' + train['prompt_question'] + ' prompt_title '+train['prompt_title']+ ' prompt_text '+train['prompt_text']
train['merged_text'].head(2)

0    text The third wave was an experimentto see ho...
1    text They would rub it up with soda to make th...
Name: merged_text, dtype: object

In [7]:
test['merged_text'] = 'text '+ test['text'] + ' prompt_question ' + test['prompt_question'] + ' prompt_title '+ test['prompt_title']+ ' prompt_text '+ test['prompt_text']
test['merged_text'].head(2)

0    text Example text 1 prompt_question Summarize....
1    text Example text 2 prompt_question Summarize....
Name: merged_text, dtype: object

In [22]:
class CFG:
    def __init__(self):
        self.model_name = '/kaggle/input/hugging-face-models-safe-tensors/bert-base-uncased'
        self.tokenizer_path = '/kaggle/input/hugging-face-models-safe-tensors/bert-base-uncased'
        self.batch_size = 16
        self.num_epochs = 40
        self.token_max = 512
        self.learning_rate = 1e-5
        self.model_save_path = '/kaggle/working/model.pt'

cfg = CFG()

In [9]:
from transformers import BertModel, BertTokenizer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [23]:
tokenizer = BertTokenizer.from_pretrained(cfg.tokenizer_path)
model_deberta = BertModel.from_pretrained(cfg.model_name)

In [13]:
from itertools import chain
from torch.utils.data import TensorDataset, DataLoader, random_split

In [24]:
def get_dataset(df):
    encoding = tokenizer(list(train['merged_text']),
                   padding=True,
                   truncation=True,
                   max_length=cfg.token_max,
                   return_tensors="pt")
    content = torch.tensor(df['content'], dtype=torch.float32)
    wording = torch.tensor(df['wording'], dtype=torch.float32)
    labels = torch.dstack((content, wording)).squeeze()
    dataset = TensorDataset(encoding['input_ids'], encoding['attention_mask'], labels)
    return dataset

ds = get_dataset(train)

In [25]:
class BERT(nn.Module):
    def __init__(self):
        super(BERT, self).__init__()
        self.bert = BertModel.from_pretrained('/kaggle/input/hugging-face-models-safe-tensors/bert-base-uncased')

        self.dropout = nn.Dropout(0.1)
        self.linear1 = nn.Linear(768, 128)
        self.linear2 = nn.Linear(128, 2) # 왜 2? content, wording이라?

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        output = self.linear1(pooled_output)
        output = nn.ReLU()(output)
        output = self.linear2(output)
        return output

In [26]:
train_ds, val_ds = random_split(ds, [0.8, 0.2])
print(len(train_ds))
print(len(val_ds))

5732
1433


In [27]:
train_dataloader = DataLoader(
            train_ds,  
            batch_size = cfg.batch_size 
)

val_dataloader = DataLoader(
            val_ds,  
            batch_size = cfg.batch_size
)

In [ ]:
# installation 
#!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev

In [ ]:
# for TPU
#import torch_xla
#import torch_xla.core.xla_model as xm

In [ ]:
#device = xm.xla_device()

In [28]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and being used")
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU instead")

GPU is available and being used


In [20]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [21]:
model = BERT().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.learning_rate)
loss_fn = nn.MSELoss() # RMSE
min_val_loss = np.inf

In [29]:
# Training loop
model.train()
patience = 0

for epoch in range(cfg.num_epochs):
    running_loss = 0.0
    for step, (input_ids, attention_mask, labels) in enumerate(train_dataloader):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        if step % 100 == 0:
            print("Epoch {}, Step {}, Loss: {}".format(epoch+1, step, loss.item()))

        running_loss += loss.item()

    print(f"Epoch {epoch+1} Loss: {running_loss / len(train_dataloader)}")

    # Validation loop
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        for val_step, (input_ids, attention_mask, labels) in enumerate(val_dataloader):
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)
            
            val_outputs = model(input_ids, attention_mask)
            val_loss += loss_fn(val_outputs, labels) 
        avg_loss = val_loss / len(val_dataloader)
        print(f"Validation Loss: {avg_loss}")
        
        if avg_loss < min_val_loss:
            patience = 0
            min_val_loss = avg_loss
            torch.save(model.state_dict(), cfg.model_save_path)
            print(f'saving model with score: {avg_loss}')
            
    patience += 1    
    if patience >= 10:
            print('Early Stopping trigerred on epoch: {}')
            break
            
    model.train()

Epoch 1, Step 0, Loss: 1.4565222263336182
Epoch 1, Step 100, Loss: 0.6990164518356323
Epoch 1, Step 200, Loss: 0.8558355569839478
Epoch 1, Step 300, Loss: 0.5735747218132019
Epoch 1 Loss: 0.5617603220896469
Validation Loss: 0.39741063117980957
saving model with score: 0.39741063117980957
Epoch 2, Step 0, Loss: 0.5681753754615784
Epoch 2, Step 100, Loss: 0.3398524522781372
Epoch 2, Step 200, Loss: 0.39584794640541077
Epoch 2, Step 300, Loss: 0.41541674733161926
Epoch 2 Loss: 0.3284636493927921
Validation Loss: 0.33838406205177307
saving model with score: 0.33838406205177307
Epoch 3, Step 0, Loss: 0.32546907663345337
Epoch 3, Step 100, Loss: 0.2607790529727936
Epoch 3, Step 200, Loss: 0.35607147216796875
Epoch 3, Step 300, Loss: 0.3234710097312927
Epoch 3 Loss: 0.2614864996092233
Validation Loss: 0.30673158168792725
saving model with score: 0.30673158168792725
Epoch 4, Step 0, Loss: 0.23219060897827148
Epoch 4, Step 100, Loss: 0.21540877223014832
Epoch 4, Step 200, Loss: 0.29507935047149

In [30]:
model = BERT()
model.load_state_dict(torch.load(cfg.model_save_path))
model.eval()


enc = tokenizer(list(test['merged_text']),
                   padding=True,
                   truncation=True,
                   max_length=cfg.token_max,
                   return_tensors="pt")

test_ds = TensorDataset(enc['input_ids'], enc['attention_mask'])
test_dataloader = torch.utils.data.DataLoader(test_ds, batch_size=16, shuffle=False)


predictions = []
with torch.no_grad():
    for input_ids, attention_mask in test_dataloader:
        input_ids = input_ids
        attention_mask = attention_mask

        outputs = model(input_ids, attention_mask)
        predictions.extend(outputs.cpu().numpy())
        

In [31]:
submission_df = pd.DataFrame({
    'student_id': test['student_id'],
    'content': [pred[0] for pred in predictions],
    'wording': [pred[1] for pred in predictions]
})

In [32]:
submission_df

,student_id,content,wording
0,000000ffffff,-0.721368,-0.901968
1,111111eeeeee,-0.656841,-0.824300
2,222222cccccc,-0.626104,-0.775375
3,333333dddddd,-0.622116,-0.767138


In [33]:
submission_df.to_csv('submission.csv', index=False)